# Series de tiempo

<p style='text-align: justify;'>Usaremos las series de tiempo para predecir el cierre de una acción en la bolsa de valores mexicana.
Podemos pensar en las series de tiempo como secuencias de datos que miden la misma cosa sobre un periodo de tiempo, podemos imaginarlo como un conjunto de datos que tiene una fecha y/o hora en la cual cada registro fue creado.
Este tipo de conjunto de datos crece constantemente, tenemos bolsas de valores que están constantemente registrando los costos, existen casas inteligentes que crean series de tiempo con la temperatura del hogar, energía entre otras medidas.</p>

<p style='text-align: justify;'>Por lo general, en las series de tiempo cada que un dato nuevo se genera, este se anexa a la serie existente. Los datos ya guardados no pueden ser modificados pues es un evento en el pasado. Además, los datos llegan ordenados por el tiempo asociado a cada registro. Podrás imaginar que el tiempo es el eje principal para este tipo de conjunto de datos.</p>

<p style='text-align: justify;'>Poder medir los sistemas presentes y guardar esa información en series de tiempo nos permite analizar el pasado, monitorear el presente y predecir el futuro.</p>

Cualquier serie de tiempo se puede describir en 4 componentes básicos:
- Nivel. El valor de referencia para la serie si fuera una línea recta.
- Tendencia. El aumento o disminución de la serie a lo largo del tiempo.
- Estacionalidad. Los patrones o ciclos a lo largo del tiempo.
- Ruido. La variabilidad en las observaciones.

Como realizar predicciones:
- Métodos Clásicos (ARIMA)
- Métodos de Machine Learning

# Series de tiempo en la bolsa de valores

https://finance.yahoo.com/quote/CEMEXCPO.MX

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv("./data/finance/CEMEXCPO.MX.csv",index_col=0)

In [25]:
# %matplotlib notebook
%matplotlib inline

In [26]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,8.975640,9.093960,8.705190,8.789700,1.790444,8140435
2000-01-04,8.451640,8.485440,8.181180,8.181180,1.666490,24190581
2000-01-05,8.189640,8.282600,8.012150,8.214990,1.673377,36945507
2000-01-06,8.231890,8.291060,7.995250,8.079760,1.645831,13890801
2000-01-07,8.113570,8.164280,8.045960,8.164280,1.663048,43027169


In [27]:
df.dtypes

Open         object
High         object
Low          object
Close        object
Adj Close    object
Volume       object
dtype: object

In [41]:
df = pd.read_csv("./data/finance/CEMEXCPO.MX.csv",index_col=0)

In [42]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,8.975640,9.093960,8.705190,8.789700,1.790444,8140435
2000-01-04,8.451640,8.485440,8.181180,8.181180,1.666490,24190581
2000-01-05,8.189640,8.282600,8.012150,8.214990,1.673377,36945507
2000-01-06,8.231890,8.291060,7.995250,8.079760,1.645831,13890801
2000-01-07,8.113570,8.164280,8.045960,8.164280,1.663048,43027169


In [43]:
df.astype(np.float32)

ValueError: could not convert string to float: 'null'

In [29]:
df.Date = pd.to_datetime(df.Date)
df.index = df.Date

AttributeError: 'DataFrame' object has no attribute 'Date'

In [ ]:
df.head(10)

In [ ]:
df = df.set_index('Date').asfreq('d')
df.head(10)

In [ ]:
df = df.fillna(method="ffill")
df.head(10)

In [ ]:
df = df["2016-06":"2018-06"]

In [ ]:
# Por un error actual en matplotlib ##########
import pandas.plotting._converter as pandacnv#
pandacnv.register()###########################
##############################################
plt.figure(figsize=(9,5))
plt.plot(df.Close);

# Haciendo predicciones a futuro
Supongamos que queremos hacer una predicción del día de mañana (actual valor a predecir). Si solo tenemos la fecha como referencia difícilmente podremos dar un pronóstico acertado pues nos falta conocer cómo se ha comportado hasta el momento la serie. Es ahí donde utilizamos ventanas deslizantes de tiempo o simplemente ventanas de tiempo, estas nos permiten crear variables artificiales las cuales nos describen el comportamiento de los días anteriores.

Esto a su ves nos permite resolver el problema de prediccion con aprendizaje supervisado donde las variables X (descripciones de la ventana) nos permite predecir una variable Y (el valor actual de cierre).
<center><img src="img/ts.png" width = "60%"></center>

In [ ]:
import seaborn as sns
import numpy as np
test_size = 60
window_size = 3

In [ ]:
df_shift = df.Close.shift(1)
df_mean_roll = df_shift.rolling(window_size).mean()
df_std_roll = df_shift.rolling(window_size).std()
df_mean_roll.name = "mean_roll"
df_std_roll.name = "std_roll"
df_mean_roll.index = df.index
df_std_roll.index = df.index

In [ ]:
df_shift.head(),df_mean_roll.head(),df_std_roll.head()

In [ ]:
df_w = pd.concat([df.Close,df_mean_roll,df_std_roll],axis=1)
df_w.head(10)

In [ ]:
df_w = df_w[window_size:]
df_w.head()

In [ ]:
test = df_w[-test_size:]
train = df_w[:-test_size]
X_test = test.drop("Close",axis = 1)
y_test = test["Close"]
X_train = train.drop("Close",axis = 1)
y_train = train["Close"]

In [ ]:
from sklearn.svm import SVR
clf = SVR(gamma="scale")
clf.fit(X_train, y_train)
y_train_hat = pd.Series(clf.predict(X_train),index=y_train.index)
y_test_hat = pd.Series(clf.predict(X_test),index=y_test.index)

In [ ]:
plt.figure(figsize=(9,5))
plt.plot(y_train ,label='Training')
plt.plot(y_test_hat,label='Test_Prediction')
plt.plot(y_test , label='Test_Real')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation');

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_test_hat)
print('MSE: {}'.format(mse))